# Meccanismi di attenzione e trasformatori

Uno dei principali svantaggi delle reti ricorrenti è che tutte le parole in una sequenza hanno lo stesso impatto sul risultato. Questo causa prestazioni subottimali con i modelli standard encoder-decoder basati su LSTM per compiti di sequenza a sequenza, come il riconoscimento di entità nominate e la traduzione automatica. In realtà, specifiche parole nella sequenza di input spesso hanno un impatto maggiore sugli output sequenziali rispetto ad altre.

Consideriamo un modello di sequenza a sequenza, come la traduzione automatica. È implementato da due reti ricorrenti, dove una rete (**encoder**) comprime la sequenza di input in uno stato nascosto, e un'altra rete, **decoder**, espande questo stato nascosto nel risultato tradotto. Il problema con questo approccio è che lo stato finale della rete ha difficoltà a ricordare l'inizio di una frase, causando una scarsa qualità del modello per frasi lunghe.

**I meccanismi di attenzione** forniscono un mezzo per pesare l'impatto contestuale di ciascun vettore di input su ciascuna previsione di output dell'RNN. Il modo in cui viene implementato è creando scorciatoie tra gli stati intermedi dell'RNN di input e l'RNN di output. In questo modo, quando si genera il simbolo di output $y_t$, si prenderanno in considerazione tutti gli stati nascosti di input $h_i$, con diversi coefficienti di peso $\alpha_{t,i}$.

![Immagine che mostra un modello encoder/decoder con uno strato di attenzione additiva](../../../../../translated_images/it/encoder-decoder-attention.7a726296894fb567.webp)
*Il modello encoder-decoder con meccanismo di attenzione additiva in [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citato da [questo post sul blog](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

La matrice di attenzione $\{\alpha_{i,j}\}$ rappresenta il grado in cui certe parole di input influenzano la generazione di una determinata parola nella sequenza di output. Di seguito è riportato un esempio di tale matrice:

![Immagine che mostra un allineamento campione trovato da RNNsearch-50, tratto da Bahdanau - arviz.org](../../../../../translated_images/it/bahdanau-fig3.09ba2d37f202a6af.webp)

*Figura tratta da [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

I meccanismi di attenzione sono responsabili di gran parte dello stato dell'arte attuale o quasi attuale nell'elaborazione del linguaggio naturale. Tuttavia, l'aggiunta di attenzione aumenta notevolmente il numero di parametri del modello, il che ha portato a problemi di scalabilità con gli RNN. Un vincolo chiave nella scalabilità degli RNN è che la natura ricorrente dei modelli rende difficile il batching e la parallelizzazione dell'addestramento. In un RNN, ogni elemento di una sequenza deve essere elaborato in ordine sequenziale, il che significa che non può essere facilmente parallelizzato.

L'adozione dei meccanismi di attenzione combinata con questo vincolo ha portato alla creazione dei modelli Transformer, ora lo stato dell'arte, che conosciamo e utilizziamo oggi, da BERT a OpenGPT3.

## Modelli Transformer

Invece di trasmettere il contesto di ciascuna previsione precedente al passo di valutazione successivo, i **modelli Transformer** utilizzano **codifiche posizionali** e attenzione per catturare il contesto di un dato input all'interno di una finestra di testo fornita. L'immagine seguente mostra come le codifiche posizionali con attenzione possano catturare il contesto all'interno di una finestra data.

![GIF animata che mostra come vengono effettuate le valutazioni nei modelli Transformer.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Poiché ogni posizione di input viene mappata indipendentemente a ciascuna posizione di output, i Transformer possono parallelizzare meglio rispetto agli RNN, il che consente modelli linguistici molto più grandi e più espressivi. Ogni testa di attenzione può essere utilizzata per apprendere diverse relazioni tra parole, migliorando i compiti di elaborazione del linguaggio naturale a valle.

**BERT** (Bidirectional Encoder Representations from Transformers) è una rete Transformer multi-strato molto grande con 12 strati per *BERT-base* e 24 per *BERT-large*. Il modello viene prima pre-addestrato su un ampio corpus di dati testuali (Wikipedia + libri) utilizzando un addestramento non supervisionato (predizione di parole mascherate in una frase). Durante il pre-addestramento, il modello acquisisce un livello significativo di comprensione del linguaggio che può poi essere sfruttato con altri dataset tramite il fine-tuning. Questo processo è chiamato **transfer learning**.

![Immagine da http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/it/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Esistono molte varianti delle architetture Transformer, tra cui BERT, DistilBERT, BigBird, OpenGPT3 e altre, che possono essere ottimizzate. Il pacchetto [HuggingFace](https://github.com/huggingface/) fornisce un repository per l'addestramento di molte di queste architetture con PyTorch.

## Utilizzo di BERT per la classificazione del testo

Vediamo come possiamo utilizzare il modello BERT pre-addestrato per risolvere il nostro compito tradizionale: la classificazione di sequenze. Classificheremo il nostro dataset originale AG News.

Per prima cosa, carichiamo la libreria HuggingFace e il nostro dataset:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Poiché utilizzeremo un modello BERT pre-addestrato, sarà necessario utilizzare un tokenizer specifico. Per prima cosa, caricheremo un tokenizer associato al modello BERT pre-addestrato.

La libreria HuggingFace contiene un repository di modelli pre-addestrati, che puoi utilizzare semplicemente specificando i loro nomi come argomenti nelle funzioni `from_pretrained`. Tutti i file binari necessari per il modello verranno scaricati automaticamente.

Tuttavia, in alcuni casi potrebbe essere necessario caricare i propri modelli. In tal caso, puoi specificare la directory che contiene tutti i file pertinenti, inclusi i parametri per il tokenizer, il file `config.json` con i parametri del modello, i pesi binari, ecc.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

L'oggetto `tokenizer` contiene la funzione `encode` che può essere utilizzata direttamente per codificare il testo:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Poi, creiamo degli iteratori che useremo durante l'addestramento per accedere ai dati. Poiché BERT utilizza la propria funzione di codifica, dovremmo definire una funzione di padding simile a `padify` che abbiamo definito in precedenza:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

Nel nostro caso, utilizzeremo il modello BERT pre-addestrato chiamato `bert-base-uncased`. Carichiamo il modello utilizzando il pacchetto `BertForSequenceClassification`. Questo garantisce che il nostro modello abbia già un'architettura necessaria per la classificazione, incluso il classificatore finale. Vedrai un messaggio di avviso che indica che i pesi del classificatore finale non sono inizializzati e che il modello richiederebbe un pre-addestramento - va benissimo, perché è esattamente ciò che stiamo per fare!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Ora siamo pronti per iniziare l'addestramento! Poiché BERT è già pre-addestrato, vogliamo partire con un tasso di apprendimento piuttosto basso per non compromettere i pesi iniziali.

Tutto il lavoro principale viene svolto dal modello `BertForSequenceClassification`. Quando applichiamo il modello ai dati di addestramento, esso restituisce sia la perdita che l'output della rete per il minibatch di input. Utilizziamo la perdita per l'ottimizzazione dei parametri (`loss.backward()` esegue il passaggio all'indietro) e `out` per calcolare l'accuratezza dell'addestramento confrontando le etichette ottenute `labs` (calcolate usando `argmax`) con le etichette attese `labels`.

Per controllare il processo, accumuliamo la perdita e l'accuratezza su diverse iterazioni e le stampiamo ogni `report_freq` cicli di addestramento.

Questo addestramento probabilmente richiederà parecchio tempo, quindi limitiamo il numero di iterazioni.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Puoi vedere (soprattutto se aumenti il numero di iterazioni e aspetti abbastanza a lungo) che la classificazione con BERT ci offre una precisione piuttosto buona! Questo perché BERT comprende già molto bene la struttura della lingua, e dobbiamo solo perfezionare il classificatore finale. Tuttavia, poiché BERT è un modello grande, l'intero processo di addestramento richiede molto tempo e necessita di una potenza computazionale significativa! (GPU, e preferibilmente più di una).

> **Note:** Nel nostro esempio, abbiamo utilizzato uno dei modelli BERT pre-addestrati più piccoli. Esistono modelli più grandi che probabilmente offrono risultati migliori.


## Valutare le prestazioni del modello

Ora possiamo valutare le prestazioni del nostro modello sul dataset di test. Il ciclo di valutazione è molto simile al ciclo di addestramento, ma non dobbiamo dimenticare di impostare il modello in modalità di valutazione chiamando `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Conclusioni

In questa unità, abbiamo visto quanto sia semplice prendere un modello linguistico pre-addestrato dalla libreria **transformers** e adattarlo al nostro compito di classificazione del testo. Allo stesso modo, i modelli BERT possono essere utilizzati per l'estrazione di entità, il question answering e altri compiti di NLP.

I modelli transformer rappresentano lo stato dell'arte attuale nel campo del NLP e, nella maggior parte dei casi, dovrebbero essere la prima soluzione con cui iniziare a sperimentare quando si implementano soluzioni NLP personalizzate. Tuttavia, comprendere i principi di base delle reti neurali ricorrenti discussi in questo modulo è estremamente importante se si desidera costruire modelli neurali avanzati.



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
